# Table of Contents

* [import libraries](#1)
* [using MultiElo to calculate changes in Elo ratings](#2)
  + [default parameter values](#2a)
* [using Player and Tracker to track ratings over time](#3)
  + [the Player object](#3a)
  + [the Tracker object](#3b)
* [handling ties](#4)
* [saving and loading ratings / batch processing](#5)
  + [deciding whether to save the rating history](#5a)
* [logging](#6)

# import libraries <a name="1"></a>

In [1]:
from multielo import MultiElo, Player, Tracker
import pandas as pd

# using `MultiElo` to calculate changes in Elo ratings <a name="2"></a>

Suppose we have a matchup where a player with an Elo rating of 1200 beats a player with an Elo rating of 1000. We can use the get_new_ratings method to calculate the new Elo ratings for those players. The ratings should be listed in the order of finish.

In [2]:
result = [1200, 1000]

elo = MultiElo()  # uses the default parameter values if nothing is supplied
elo.get_new_ratings(result)

array([1207.68809835,  992.31190165])

We can pass different parameter values to the MultiElo object if we don't want to use the default values.

In [3]:
elo_custom = MultiElo(k_value=64, d_value=800)
elo_custom.get_new_ratings(result)

array([1223.03584001,  976.96415999])

We can also use the get_expected_scores method to get the expected scores for each player (in 1-on-1 matchups, this can be interpreted as the predicted win probability).

In [4]:
elo.get_expected_scores(result)

array([0.75974693, 0.24025307])

We can calculate expected scores and new Elo ratings for multiplayer matchups using the same `MultiElo` object. The methodology behind this implementation of multiplayer Elo is described in the README. In this four-player example, a player with a 1200 rating comes in 1st, 1000 comes in second, 800 comes in third, and 900 comes in last.

In [5]:
multiplayer_result = [1200, 1000, 800, 900]

In [6]:
elo.get_new_ratings(multiplayer_result)

array([1207.71426754, 1005.75896   ,  804.94244537,  881.58432708])

In [7]:
# the expected scores are less interpretable than the 1-on-1 case
elo.get_expected_scores(multiplayer_result)

array([0.41964305, 0.27334417, 0.11518286, 0.19182993])

Given the ratings of all players involved in a game, we can estimate the probability of each player finishing in each possible place (1st, 2nd, ..., last). **Note:** This calculation is done using a simulation so the probabilities will not be exact and they may change if you use a different random seed.

In [8]:
# (i, j) value is probability that player i finishes in place j
elo.simulate_win_probabilities([1200, 1000, 800, 900])

array([[0.62369, 0.27832, 0.08497, 0.01302],
       [0.19969, 0.37015, 0.30186, 0.1283 ],
       [0.06344, 0.1291 , 0.24257, 0.56489],
       [0.11318, 0.22243, 0.3706 , 0.29379]])

## default parameter values <a name="2a"></a>

As alluded to above, Elo has a few parameters that need to be set. I picked some arbitrary but pretty standard values. (Except the scoring function -- that isn't part of standard 1-vs-1 Elo.)

The parameters are discussed in more detail in the README.

In [9]:
from multielo.multielo import DEFAULT_K_VALUE, DEFAULT_D_VALUE, DEFAULT_SCORING_FUNCTION_BASE

print(f"DEFAULT_K_VALUE: {DEFAULT_K_VALUE}")
print(f"DEFAULT_D_VALUE: {DEFAULT_D_VALUE}")
print(f"DEFAULT_SCORING_FUNCTION_BASE: {DEFAULT_SCORING_FUNCTION_BASE}")

DEFAULT_K_VALUE: 32
DEFAULT_D_VALUE: 400
DEFAULT_SCORING_FUNCTION_BASE: 1


Given the ratings of all players involved in a game, we can estimate the probability of each player finishing in each possible place (1st, 2nd, ..., last). **Note:** This calculation is done using a simulation so the probabilities will not be exact and they may change if you use a different random seed.

In [10]:
# (i, j) value is probability that player i finishes in place j
elo.simulate_win_probabilities([1200, 1000, 800, 900])

array([[0.6271 , 0.27717, 0.08288, 0.01285],
       [0.19932, 0.37078, 0.30143, 0.12847],
       [0.06253, 0.12959, 0.24625, 0.56163],
       [0.11105, 0.22246, 0.36944, 0.29705]])

# using `Player` and `Tracker` to track ratings over time <a name="3"></a>

The package includes two objects -- `Player` and `Tracker` -- that make it easy to track Elo ratings over time. For example, if you have a league with players or teams that play each other multiple times, the `Tracker` object is useful.

## the `Player` object <a name="3a"></a>

This object track's a single player's Elo rating over time. Each player requires a player ID and you can supply an initial rating or the default value will be used.

In [11]:
player_default = Player("player_A")
player_custom = Player("player_B", rating=1200)

In [12]:
player_default, player_custom

(Player(id = player_A, rating = 1000.00, n_games = 0),
 Player(id = player_B, rating = 1200.00, n_games = 0))

We can update a player's rating with the update_rating method. Specifying a date will record the change as a game result. Dates can be any sortable value such that sorting in ascending order will order the dates from earliest to latest -- strings in "YYYY-MM-DD" format or integers work well.

In [13]:
player_default.update_rating(1050, date="2020-07-01")
player_default.update_rating(1075, date="2020-07-15")

player_default

Player(id = player_A, rating = 1075.00, n_games = 2)

The rating_history attribute displays the Elo rating history for the player. Each entry is a (date, rating) tuple. The first entry is assumed to be an initial rating before any games were played and all subsequent entries will be counted as games.

In [14]:
player_default.rating_history

[(None, 1000), ('2020-07-01', 1050), ('2020-07-15', 1075)]

In [15]:
player_default.count_games()

2

We can also obtain the player's rating as of any date using the get_rating_as_of_date method.

In [16]:
player_default.get_rating_as_of_date("2020-07-05")

1050

## the `Tracker` object <a name="3b"></a>

This object stores multiple `Player` objects and updates the ratings of those players as matchups occur. The tracker is useful for processing many matchup matchup results and viewing the historical Elo ratings of all players in the tracker (e.g., all players/teams in a league).

A pandas dataframe of results is required for the tracker. The dataframe must have a column for the date of the matchup and columns for the places that players finished in for that matchup. The place columns can be named anything, but it is assumed that 1st to last goes from the leftmost column to the rightmost column. Note that there can be a different number of players in each matchup.

In [17]:
data = pd.DataFrame({
    "date": ["2020-03-29", "2020-04-05", "2020-04-12", "2020-04-19"],
    "1st": ["Homer", "Lisa", "Lisa", "Marge"],
    "2nd": ["Marge", "Bart", "Marge", "Lisa"],
    "3rd": ["Bart", "Homer", "Homer", None],
    "4th": [None, None, "Bart", None]
})

data

,date,1st,2nd,3rd,4th
0,2020-03-29,Homer,Marge,Bart,None
1,2020-04-05,Lisa,Bart,Homer,None
2,2020-04-12,Lisa,Marge,Homer,Bart
3,2020-04-19,Marge,Lisa,None,None


The process_data method is used to calculate the current and historical ratings for all players. The tracker will loop through the supplied dataframe from the earliest to latest date and update Elo for all players involved in each matchup. New `Player` objects will be created when a player ID (the name in the dataframe) is encountered for the first time. 

In [18]:
tracker = Tracker()
tracker.process_data(data)

After processing the data we can easily view the current and historical ratings for all players.

In [19]:
tracker = Tracker()
tracker.process_data(data)

In [20]:
tracker.get_current_ratings()

,rank,player_id,n_games,rating
0,1,Lisa,3,1025.750704
1,2,Marge,3,1025.623287
2,3,Homer,3,990.222321
3,4,Bart,3,958.403688


In [21]:
tracker.get_history_df()

,player_id,date,rating
0,Homer,2020-03-29,1021.333333
1,Homer,2020-04-05,998.042495
2,Homer,2020-04-12,990.222321
3,Marge,2020-03-29,1000.000000
4,Marge,2020-04-12,1007.999846
5,Marge,2020-04-19,1025.623287
6,Bart,2020-03-29,978.666667
7,Bart,2020-04-05,980.624172
8,Bart,2020-04-12,958.403688
9,Lisa,2020-04-05,1021.333333


It is also easy to plot the Elo rating history of all players over time.

In [22]:
import altair as alt

df = tracker.get_history_df()
alt.Chart(df).mark_line(point=True).encode(
    x="date:T",
    y=alt.Y("rating:Q", scale=alt.Scale(zero=False)),
    color="player_id:N",
)

alt.Chart(...)

By default, the `Tracker` object uses a `MultiElo` object with all default settings to calculate changes in Elo rating. The user can alternatively pass in a `MultiElo` object with any settings, which will affect the results. For example, larger K means rankings change more in each individual matchup.

In [23]:
k = 128
bigk_elo = MultiElo(k_value=k)
bigk_tracker = Tracker(elo_rater=bigk_elo)
bigk_tracker.process_data(data)

bigk_tracker.get_current_ratings()

,rank,player_id,n_games,rating
0,1,Marge,3,1117.029251
1,2,Lisa,3,1065.955505
2,3,Homer,3,948.872182
3,4,Bart,3,868.143061


The chosen scoring function has a large effect on the rankings because it determines the relative value of finishing first, second, third, etc. The default scoring function (the "linear score function") used in the example above doesn't provide proportionally more value to finishing first (i.e., moving from 6th to 5th is as good as improving from 2nd to 1st).

To provide more value to the top finishers, we can use an "exponential score function". The example below uses an exponential score function with a base of 1.5. Note that the final rating values are different from the example using the linear score function above.

In [24]:
exp_elo = MultiElo(score_function_base=1.5)
exp_tracker = Tracker(elo_rater=exp_elo)
exp_tracker.process_data(data)

exp_tracker.get_current_ratings()

,rank,player_id,n_games,rating
0,1,Lisa,3,1035.161374
1,2,Marge,3,1020.663059
2,3,Homer,3,988.457875
3,4,Bart,3,955.717692


# handling ties <a name="4"></a>

As of version 0.3.0, this Elo implementation can handle ties. The syntax to annotate ties in the `MultiElo` and `Tracker` objects is shown below.

In `MultiElo`, use the result_order parameter to indicate which place each player finished in, where lower indicates a better finishing position. (Note: the exact values do not matter -- they just need to be increasing)

In [25]:
# first player beat the second in a two-player matchup (default)
elo.get_new_ratings([1200, 1000], result_order=[1, 2])

array([1207.68809835,  992.31190165])

In [26]:
# two players tied in a two-player matchup
elo.get_new_ratings([1200, 1000], result_order=[1, 1])

array([1191.68809835, 1008.31190165])

In [27]:
# tie for first place in three-player matchup
elo.get_new_ratings([1200, 1000, 800], result_order=[1, 1, 2])

array([1196.39812617, 1010.66666667,  792.93520716])

In [28]:
# tie for last place in three-player matchup
elo.get_new_ratings([1200, 1000, 800], result_order=[1, 2, 2])

array([1207.06479284,  989.33333333,  803.60187383])

In [29]:
# three-way tie in three-player matchup
elo.get_new_ratings([1200, 1000, 800], result_order=[1, 1, 1])

array([1185.7314595, 1000.       ,  814.2685405])

# saving and loading ratings / batch processing <a name="5"></a>

It is possible save and load (or export and import) ratings from a `Tracker` object to allow for a "batch processing" workflow:
1. Process data from the first time period.
2. Export current ratings for all players.
3. When new data is available some time later, import the current ratings and process *only the new data* (rather than all data from the beginning).
4. Repeat as needed.

In [30]:
tracker = Tracker()
tracker.process_data(data)  # data was defined previously

# save the data (use whatever file extension you want... it's technically a pickle file)
tracker.save_player_data("./example_player_data.elo")

In [31]:
# come back later when we have new data...
new_data = pd.DataFrame({
    "date": ["2020-05-01", "2020-05-08"],
    "1st": ["Ned", "Lisa"],
    "2nd": ["Marge", "Marge"],
    "3rd": ["Homer", "Ned"],
    "4th": [None, "Homer"],
})

# load the current ratings into a new Tracker object
new_tracker = Tracker(players="./example_player_data.elo")

# process the new data
new_tracker.process_data(new_data)

new_tracker.get_current_ratings()

,rank,player_id,n_games,rating
0,1,Lisa,4,1048.347519
1,2,Marge,5,1030.534457
2,3,Ned,2,1012.775331
3,4,Bart,3,958.403688
4,5,Homer,5,949.939005


## deciding whether to save the rating history <a name="5a"></a>

Depending on your use case, you may or may not care about the full Elo rating history for each player. By default, the Player and Tracker objects save the full rating history. You may want to disable this behavior for large datasets in order to save memory (and disk space if you are exporting ratings).

You can disable the rating history when creating a Tracker object by setting `keep_history=False`. Similarly, when exporting ratings you can set `save_full_history=False`.

In [32]:
tracker = Tracker(keep_history=False)  # this tracker won't save the full rating history of the players

tracker.save_player_data("/dev/null", save_full_history=False)  # this will only export the current ratings

# logging <a name="6"></a>

The `MultiElo` and `Tracker` objects have loggers that output information about the computations. Set the loggers to INFO or DEBUG for verbose output.

In [33]:
import logging

logging.basicConfig()
logging.getLogger("multielo").setLevel(logging.INFO)

In [34]:
elo = MultiElo()
tracker = Tracker(elo)
tracker.process_data(data)
tracker.get_current_ratings()

INFO:multielo.player_tracker:Created Tracker with Elo parameters K=32, D=400
INFO:multielo.player_tracker:created player with ID Homer and rating 1000
INFO:multielo.player_tracker:created player with ID Marge and rating 1000
INFO:multielo.player_tracker:created player with ID Bart and rating 1000
INFO:multielo.player_tracker:processing rating changes for date 2020-03-29...
INFO:multielo.player_tracker:Updating rating for Homer: 1000.000 --> 1021.333
INFO:multielo.player_tracker:Updating rating for Marge: 1000.000 --> 1000.000
INFO:multielo.player_tracker:Updating rating for Bart: 1000.000 --> 978.667
INFO:multielo.player_tracker:created player with ID Lisa and rating 1000
INFO:multielo.player_tracker:processing rating changes for date 2020-04-05...
INFO:multielo.player_tracker:Updating rating for Lisa: 1000.000 --> 1021.333
INFO:multielo.player_tracker:Updating rating for Bart: 978.667 --> 980.624
INFO:multielo.player_tracker:Updating rating for Homer: 1021.333 --> 998.042
INFO:multiel

,rank,player_id,n_games,rating
0,1,Lisa,3,1025.750704
1,2,Marge,3,1025.623287
2,3,Homer,3,990.222321
3,4,Bart,3,958.403688
